In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [103]:
pip install --upgrade pandas

Note: you may need to restart the kernel to use updated packages.


In [104]:
import requests
import numpy as np
import pandas as pd

In [183]:
# Load the data
with open("stations_list.txt", "r") as fh:
    trainingSet = [] # Training set data
    splitNumbers = []
    for numbers in fh:
        # Split string into list of substrings
        splitNumbers = numbers.split()
        trainingSet.append(splitNumbers)

arr_int = np.array(trainingSet)
arr_flat = arr_int.flatten().astype(int)

sensor_ids = list(arr_flat)

my_list = list(range(402091, 402706)) 
sensor_ids.extend(my_list)

print(sensor_ids)

In [109]:
# Replace these with your PeMS credentials
username = "USERNAME"
password = "PASSWORD"

# URL for logging into PeMS
login_url = "https://pems.dot.ca.gov/"

# Time parameters (same for all sensor IDs)
s_time_id = "1536537600"
e_time_id = "1537142340"
s_time_id_f = "09%2F10%2F2018+00%3A00"
e_time_id_f = "09%2F16%2F2018+23%3A59"

# Start a session
session = requests.Session()

# Log in to PeMS
login_payload = {
    'username': username,
    'password': password,
    'submit': 'Login'
}

login_response = session.post(login_url, data=login_payload)

if login_response.status_code == 200 and "logout" in login_response.text.lower():
    print("Login successful!")

       sensor_id in sensor_ids:
        # Format the data URL with the current sensor ID
        data_url = f"https://pems.dot.ca.gov/?report_form=1&dnode=VDS&content=loops&tab=det_timeseries&export=text&station_id={sensor_id}&s_time_id={s_time_id}&s_time_id_f={s_time_id_f}&e_time_id={e_time_id}&e_time_id_f={e_time_id_f}&tod=all&tod_from=0&tod_to=0&dow_0=on&dow_1=on&dow_2=on&dow_3=on&dow_4=on&dow_5=on&dow_6=on&q=flow&q2=&gn=5min&agg=on&lane1=on&lane2=on&lane3=on&lane4=on"
        
        response = session.get(data_url)

        if response.status_code == 200:

            data_lines = response.text.splitlines()
            filtered_data = []

            filtered_data.append(data_lines[0])

            for line in data_lines[1:]:
                columns = line.split("\t") 
                columns = [col.strip() for col in columns]
                time_column = columns[0]  

                try:
                    time_part = time_column.split(" ")[1]  # Get time after the date
                    minutes = int(time_part.split(":")[1])

                    if minutes % 10 == 0:
                        filtered_data.append(line)
                except ValueError:
                    print(f"Skipping invalid time format in line: {line}")

            filename = f"occupancy_data_{sensor_id}_filtered.txt"  # Filename without path

            with open(filename, "w") as file:
                file.write("\n".join(filtered_data))
        else:
            print(f"Failed to fetch data for sensor {sensor_id}. Status code: {response.status_code}")
    print("Done")
else:
    print("Login failed. Please check your credentials.")



Login successful!
Done


In [ ]:
import pandas as pd
import os

for sensor_id in sensor_ids:
    input_file = f"occupancy_data_{sensor_id}_filtered.txt"  # Filename without path
    df = pd.read_csv(input_file, delimiter="\t")  # Adjust delimiter if needed
    df['5 Minutes'] = pd.to_datetime(df['5 Minutes'], format='%m/%d/%Y %H:%M')
    df['Date'] = df['5 Minutes'].dt.date  # Extract the date part
    grouped = df.groupby('Date')['Flow (Veh/5 Minutes)'].apply(list).reset_index()
    grouped['Flow List'] = grouped['Flow (Veh/5 Minutes)'].apply(lambda x: ' '.join(map(str, x)) + ' ')
    grouped = grouped.drop(columns=['Date', 'Flow (Veh/5 Minutes)'])
    output_file = f"filtered_occupancy_data_{sensor_id}_filtered.txt"  # Filename without path
    with open(output_file, 'w') as f:
        for row in grouped['Flow List']:
            f.write(row + '\n')

# Delete the original file after saving the new file
    if os.path.exists(input_file):
        os.remove(input_file)
        print(f"Original file {input_file} has been deleted.")
    else:
        print(f"The original file {input_file} does not exist.")

In [185]:
# Load the two files
# Create initial seed file
seed_Sensor_ID1 = 400000
seed_Sensor_ID2 = 400001
file1_path = f"filtered_occupancy_data_{seed_Sensor_ID1}_filtered.txt"  # Filename without path
file2_path = f"filtered_occupancy_data_{seed_Sensor_ID2}_filtered.txt"  # Filename without path
#    file2_path = f"filtered_occupancy_data_{sensor_id}_filtered.txt"  # Filename without path
output_path = 'appended_output_seed.txt'

# Check if both files exist
if not os.path.exists(file1_path):
    raise FileNotFoundError(f"File not found: {file1_path}")

if not os.path.exists(file2_path):
    print(f"File not found: {file2_path}. Skipping file2 and continuing with file1 only.")
    file2_path = None


with open(file1_path, 'r') as file1:
    lines1 = file1.readlines()


if file2_path:
    with open(file2_path, 'r') as file2:
        lines2 = file2.readlines()


    if len(lines1) != len(lines2):
        raise ValueError("The files do not have the same number of rows.")

    appended_lines = [line1.strip() + line2.strip() + '\n' for line1, line2 in zip(lines1, lines2)]
else:
    appended_lines = [line1.strip() + '\n' for line1 in lines1]
    
with open(output_path, 'w') as output_file:
    output_file.writelines(appended_lines)

In [ ]:
numbers_to_remove = [400000, 400001] 

sensor_ids = [x for x in sensor_ids if x not in numbers_to_remove]

print(sensor_ids)

In [ ]:
# Load the two files
for sensor_id in sensor_ids:

    file1_path = 'appended_output_seed.txt'
    file2_path = f"filtered_occupancy_data_{sensor_id}_filtered.txt"  
    output_path = 'appended_output_seed.txt'

    if not os.path.exists(file1_path):
        raise FileNotFoundError(f"File not found: {file1_path}")

    if not os.path.exists(file2_path):
        print(f"File not found: {file2_path}. Skipping file2 and continuing with file1 only.")
        file2_path = None

    with open(file1_path, 'r') as file1:
        lines1 = file1.readlines()

    if file2_path:
        with open(file2_path, 'r') as file2:
            lines2 = file2.readlines()

        if len(lines1) != len(lines2):
            raise ValueError("The files do not have the same number of rows.")

        appended_lines = [line1.strip() + line2.strip() + '\n' for line1, line2 in zip(lines1, lines2)]
    else:
        appended_lines = [line1.strip() + '\n' for line1 in lines1]

    with open(output_path, 'w') as output_file:
        output_file.writelines(appended_lines)

In [ ]:
# Load the training data
with open("appended_output_seed.txt", "r") as fh:
    testSet = [] # Training set data
    splitNumbers = []
    # Using a for loop to convert each line of strings into a row of floats
    for numbers in fh:
        # Split string into list of substrings
        splitNumbers = numbers.split()
        testSet.append(splitNumbers)

# Set in np to reduce computation time in calculations
np_Test_Set = np.array(testSet, dtype=np.float64)
print(f'Shape of training data: {np_Test_Set.shape}')

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

tf = TfidfTransformer() # weight word significance
nl = Normalizer(copy=False) # scale each data point
pl = make_pipeline(tf, nl)
Transform_Norm_Test_Data = pl.fit_transform(np_Test_Set)
print("Done")

In [ ]:
dense_data = Transform_Norm_Test_Data.toarray()

# Save to a text file
output_file = 'self_test.txt'
np.savetxt(output_file, dense_data, fmt='%0.5f')  # Save with 5 decimal places

print(f"Transformed data saved to {output_file}")